## Anti-bestemmie function

Actually checkpoint this shit. Not just **one** checkpoint, but as many as you wish.

In [1]:
import os
import datetime
import shutil


def checkpoint(notebook_name):
    idx = os.listdir().index(notebook_name + '.ipynb')
    name = str(datetime.datetime.now())
    shutil.copy('./' + notebook_name + '.ipynb', './.ipynb_checkpoints/' + notebook_name + '-' + name + '.ipynb')

checkpoint('analysis')

## Actual work

In [2]:
import sys

wd = os.getcwd() + '/'
sys.path.append(wd + '../')
sys.path.append(wd + '../../')

import numpy as np
import pandas as pd
import json

from models import Rule

from numpy import array
from statistics import harmonic_mean

# Auto-reload code
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Configurations

Configurations to run the analysis. They include:
- hyperparameters
- input folders/files
- output folders/files

In [3]:
datasets = ['adult', 'churn', 'compas', 'german']
black_boxes = ['dnn', 'rf']

# Logs appendices with template
# {$dataset}.{$black box}.{$scoring}.results.json
logs_template = '{0}.{1}.{2}.results.json'
# {$dataset}.{$adversary}.{$black_box}.{$scoring}.json
adversary_logs_template = '{0}.{1}.{2}.{3}.results.json'

## Load
Load data for the given configuration.

In [4]:
def load(dataset, wp, ws, alpha, beta, maxlen):
    template = logs_template.format(dataset)

    with open(wd + '../output/' + template, 'r') as log:
        results = json.load(log)

    run = None
    for entry in results['runs']:
        if (entry['coverage'], entry['sparsity'], entry['alpha'], entry['beta'], entry['max_len']) == (wp, ws, alpha, beta, maxlen):
            run = entry
            break
    if entry is None:
        raise ValueError('Entry not found: ' + str((dataset, wp, ws, alpha, beta, maxlen)))

    return entry

## Adversaries

The adversary set is comprised of:

- Decision Tree (DT)
- Pruned Decision Tree (DT)
- CPAR
- FOIL
- Trepan
- CORELS
- SBRL (Scalable Bayesian Rule Lists)

Validation files **have already been computed**, they are in `./{$dataset}-validation.csv`.

In [5]:
adversaries = ['corels', 'cpar', 'sbrl']

# Plotting

## Data loading

In [6]:
results_per_bb = {}
black_boxes = 'dnn', 'rf'
# Data columns
meta_cols = ['dataset', 'black_box', 'scoring', 'k']
cols = ['alpha',
        'beta',
        'gamma',
        'max_len',
        'scoring-fidelities',
        'beta-scoring-fidelity', 'beta-scoring-rule_nr',
        'coverage', 'beta-scoring-coverage',
        'mean_length', 'std_length',
        'mean-beta-scoring-length', 'std-beta-scoring-length',
        'mean_prediction', 'std-scoring-prediction',
        'rule_reduction-beta-scoring', 'len_reduction-beta-scoring','simplicity-beta-scoring',
         'feature_frequency', 'scoring-beta-feature_frequency'
       ]

# Baseline results
for bb in black_boxes:
    results = []
    for dataset in datasets:
        for scoring_foo in ['r2', 'coverage', 'fidelity']:
            template = logs_template.format(dataset, bb, scoring_foo)

            with open(wd + '../output/' + template, 'r') as log:
                dataset_results = json.load(log)

            df = pd.concat([pd.DataFrame({k: run['results'][k] for k in cols})
                            for run in dataset_results['runs']], axis='rows')
            df['k'] = [1 for _ in range(df.shape[0])]
            df['dataset'] = dataset
            df['black_box'] = bb
            df['scoring'] = scoring_foo
            df = df[meta_cols + cols]
            results += [df]

    # Full DataFrame construction (Warning, may be slow)
    results = pd.concat(results, axis='rows', ignore_index=True)
    results_per_bb[bb] = results

full_results = pd.concat(list(results_per_bb.values()), axis='rows').drop_duplicates()
full_results = full_results.sort_values(['dataset', 'black_box', 'beta'])


# Adversaries
missing = []
results_per_bb = {}
for bb in ['rf', 'dnn']:
    results = []
    for adversary in adversaries:
        for dataset in datasets:
            for scoring_foo in ['r2', 'coverage', 'fidelity']:
                try:
                    if dataset == 'german' and adversary == 'corels':
                        continue

                    template = adversary_logs_template.format(dataset, adversary, bb, scoring_foo)

                    with open(wd + '../output/' + template, 'r') as log:
                        dataset_results = json.load(log)

                    df = pd.concat([pd.DataFrame({k: run['results'][k] for k in cols})
                                    for run in dataset_results['runs']], axis='rows')
                    df['k'] = [1 for _ in range(df.shape[0])]
                    df['dataset'] = dataset
                    df['black_box'] = bb
                    df['scoring'] = scoring_foo
                    df['algorithm'] = adversary
                    df = df[meta_cols + ['algorithm'] + cols]
                    results += [df]
                except FileNotFoundError as e:
                    missing.append((dataset, adversary, bb, scoring_foo))

    # Full DataFrame construction (Warning, may be slow)
    results = pd.concat(results, axis='rows', ignore_index=True)
    results_per_bb[bb] = results

full_adversaries_results = pd.concat(list(results_per_bb.values()), axis='rows').drop_duplicates()
full_adversaries_results = full_adversaries_results[full_adversaries_results.alpha != 0]
full_adversaries_results = full_adversaries_results.sort_values(['dataset', 'black_box', 'beta'])

## Plotting utils

In [7]:
# Plotting utils
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, LabelSet, Legend, Range1d, ColorBar
from bokeh.models import SingleIntervalTicker, LinearAxis, NumeralTickFormatter
from bokeh.models.tickers import ContinuousTicker

from bokeh.io import output_notebook, output_file
from bokeh.io import export_svgs, export_png

# Output in the notebook instead of file
# output_notebook()

# Color palettes
from bokeh.palettes import RdYlGn11 as plt
from bokeh.transform import linear_cmap

RED   = '#F05F54'
BLUE  = '#3169A3'
BLACK = '#000000'

shapes = ['square', 'circle', 'triangle', 'diamond']
plt_colors4 = ['#f16a70', '#b1d877', '#4d4d4d', '#8cdcda']
plt_colors3 = ['#BE4A2F', '#225E78', '#72AD2B']
plt_colors3 = ['#009988', '#004488', '#bb5566', '#000000', '#ddaa33']

In [8]:
summary_columns = ['mean', 'std', '25%', '50%', '75%', 'min', 'max']
prediction_cols = ['mean_prediction', 'std-scoring-prediction']

## Plotting functions: Global models

In [9]:
def global_plot_fidelity_on_beta_cut(measure=None, scoring='r2', black_box=None, draw_left_axis_label=True, name=None):
    if measure == 'fidelity':
        measure_name = 'beta_fidelity_fidelities'
    else:
        measure_name = 'beta_scoring_fidelity'
        
    # Data gathering
    if isinstance(black_box, str):
        df = full_adversaries_results[(full_adversaries_results.black_box == black_box) & (full_results.alpha == .5)]
        df = df.groupby(['dataset', 'beta'])[measure_name].describe()[summary_columns]
    else:
        df = full_adversaries_results[(full_adversaries_results.black_box.isin(black_box)) & (full_results.alpha == .5)]
        df = df.groupby(['dataset', 'beta'])[measure_name].describe()[summary_columns]

    # Plot
    p = figure(title='Mean fidelity, free β', x_axis_type='linear', x_minor_ticks=5, plot_width=900, plot_height=600)
    p.title.text_font_size = '20pt'

    # Axes
    p.xaxis[0].ticker.desired_num_ticks = 10
    p.yaxis[0].ticker.desired_num_ticks = 5
    p.xaxis[0].axis_label_text_font_size = '20pt'
    p.yaxis[0].axis_label_text_font_size = '20pt'
    p.xaxis[0].major_label_text_font_size = '20pt'
    p.yaxis[0].major_label_text_font_size = '20pt'
    p.xaxis.axis_label = 'β'
    p.yaxis.axis_label = 'fidelity'

    # Plot borders
    left, right, bottom, top = 0, 104, .75, 1.09
    p.x_range = Range1d(left, right)
    p.y_range = Range1d(bottom, top)

    for d, shape, color in zip(datasets, shapes, plt_colors4):
        dataset_df = df.loc[d]
        
        # Fill nan data
        nan_index_start = np.argwhere(~dataset_df['mean'].notnull()).flatten().squeeze()
        nan_index_start = nan_index_start.item(0) if nan_index_start.size > 0 else dataset_df.shape[0]
        dataset_df.loc[~dataset_df['mean'].notnull(), 'mean'] = dataset_df[dataset_df['mean'].notnull()].iloc[-1]['mean']
        
        x, y = dataset_df.index.values.tolist(), dataset_df['mean'].values
        argmax_y = np.where(y == y.max())[0][-1]

        # Data labels
        source = ColumnDataSource(data=dict(x=x, y=y, names=[str(y_)[:4] for y_ in y]))
        computed_source = ColumnDataSource(data=dict(x=x, y=y))
        best_source = ColumnDataSource(data=dict(x=[x[argmax_y]], y=[y[argmax_y]]))

        # Available data
        line = p.line(x='x', y='y', line_width=4, alpha=.5,
                      line_color=color, source=computed_source, line_dash='dashed')
        dots = getattr(p, shape)(x='x', y='y', size=16, line_width=2, color=color,
                                 source=computed_source, legend=d)
    # Legend
    p.legend.location = 'top_left'
    p.legend[0].orientation = 'horizontal'
    p.legend.label_text_font_size = '20pt'

    if name is None:
        show(p)
    else:
        p.output_backend = 'svg'
        export_svgs(p, name)
        return p


def global_plot_coverage_on_beta_cut(measure=None, black_box=None, draw_left_axis_label=True, name=None, legend_position='bottom_left', legend_orientation='vertical'):
    if measure == 'fidelity':
        measure_name = 'beta-fidelity-coverage'
    else:
        measure_name = 'beta-scoring-coverage'
        
    # Data gathering
    if isinstance(black_box, str):
        df = full_adversaries_results[(full_adversaries_results.black_box == black_box) & (full_results.alpha == .5)]
        df = df.groupby(['dataset', 'beta'])[measure_name].describe()[summary_columns]
    else:
        df = full_adversaries_results[(full_adversaries_results.black_box.isin(black_box)) & (full_results.alpha == .5)]
        df = df.groupby(['dataset', 'beta'])[measure_name].describe()[summary_columns]

    # Plot
    p = figure(title='Mean coverage, free β', x_axis_type='linear', x_minor_ticks=5, plot_width=900, plot_height=600)
    p.title.text_font_size = '20pt'

    # Axes
    p.xaxis[0].ticker.desired_num_ticks = 10
    p.yaxis[0].ticker.desired_num_ticks = 5
    p.xaxis[0].axis_label_text_font_size = '20pt'
    p.yaxis[0].axis_label_text_font_size = '20pt'
    p.xaxis[0].major_label_text_font_size = '20pt'
    p.yaxis[0].major_label_text_font_size = '20pt'
    p.xaxis.axis_label = 'β'
    p.yaxis.axis_label = 'coverage'

    # Plot borders
    left, right, bottom, top = 0, 109, 0, 1.09
    p.x_range = Range1d(left, right)
    p.y_range = Range1d(bottom, top)

    for d, shape, color in zip(datasets, shapes, plt_colors4):
        dataset_df = df.loc[d]
        
        # Fill nan data
        nan_index_start = np.argwhere(~dataset_df['mean'].notnull()).flatten().squeeze()
        nan_index_start = nan_index_start.item(0) if nan_index_start.size > 0 else dataset_df.shape[0]
        dataset_df.loc[~dataset_df['mean'].notnull(), 'mean'] = dataset_df[dataset_df['mean'].notnull()].iloc[-1]['mean']
        
        x, y = dataset_df.index.values.tolist(), dataset_df['mean'].values
        argmax_y = np.where(y == y.max())[0][-1]

        # Data labels
        source = ColumnDataSource(data=dict(x=x, y=y, names=[str(y_)[:4] for y_ in y]))
        computed_source = ColumnDataSource(data=dict(x=x, y=y))
        best_source = ColumnDataSource(data=dict(x=[x[argmax_y]], y=[y[argmax_y]]))

        # Available data
        line = p.line(x='x', y='y', line_width=4, alpha=.5,
                      line_color=color, source=computed_source, line_dash='dashed')
        dots = getattr(p, shape)(x='x', y='y', size=16, line_width=2, color=color,
                                 source=computed_source, legend=d)
    # Legend
    p.legend.location = legend_position
    p.legend[0].orientation = legend_orientation
    p.legend.label_text_font_size = '20pt'

    if name is None:
        show(p)
    else:
        p.output_backend = 'svg'
        export_svgs(p, name)
        return p

## Plotting functions: Baseline

In [10]:
def plot_fidelity_on_beta_cut(scoring='r2', black_box=None, draw_left_axis_label=True, name=None):    
    # Data gathering
    if isinstance(black_box, str):
        df = full_results[(full_results.black_box == black_box) & (full_results.scoring == scoring)\
                         & (full_results.alpha == .5)]
    else:
        df = full_results[(full_results.black_box.isin(black_box)) & (full_results.scoring == scoring)\
                         & (full_results.alpha == .5)]

    # Plot
    scoring = 'r²' if scoring == 'r2' else scoring
    p = figure(title='[' + scoring + '] Fidelity [' + black_box.upper() + ']', x_axis_type='linear',
               x_minor_ticks=5, plot_width=600, plot_height=500)
    p.title.text_font_size = '20pt'

    # Axes
    p.xaxis[0].ticker.desired_num_ticks = 10
    p.yaxis[0].ticker.desired_num_ticks = 5
    p.xaxis[0].axis_label_text_font_size = '25pt'
    p.yaxis[0].axis_label_text_font_size = '25pt' if draw_left_axis_label else '0pt'
    p.xaxis[0].major_label_text_font_size = '20pt'
    p.yaxis[0].major_label_text_font_size = '20pt' if draw_left_axis_label else '0pt'
    p.xaxis.axis_label = 'β'
    # p.yaxis.visible = draw_left_axis_label
    p.yaxis.axis_label = 'fidelity' if draw_left_axis_label else ''

    # Plot borders
    left, right, bottom, top = 0, 104, .5, 1.09
    p.x_range = Range1d(left, right)
    p.y_range = Range1d(bottom, top)

    for d, shape, color in zip(datasets, shapes, plt_colors4):
        dataset_df = df[df.dataset == d]
        
        # Fill nan data
        nan_index_start = np.argwhere(~dataset_df['beta-scoring-fidelity'].notnull()).flatten().squeeze()
        nan_index_start = nan_index_start.item(0) if nan_index_start.size > 0 else dataset_df.shape[0]
        dataset_df.loc[~dataset_df['beta-scoring-fidelity'].notnull(), 'beta-scoring-fidelity'] = dataset_df[dataset_df['beta-scoring-fidelity'].notnull()].iloc[-1]['beta-scoring-fidelity']
        
        x, y = dataset_df.beta.values, dataset_df['beta-scoring-fidelity'].values
        argmax_y = np.where(y == y.max())[0][-1]

        # Data labels
        source = ColumnDataSource(data=dict(x=x, y=y, names=[str(y_)[:4] for y_ in y]))
        computed_source = ColumnDataSource(data=dict(x=x, y=y))
        best_source = ColumnDataSource(data=dict(x=[x[argmax_y]], y=[y[argmax_y]]))

        # Available data
        line = p.line(x='x', y='y', line_width=4, alpha=.5,
                      line_color=color, source=computed_source, line_dash='dashed')
        dots = getattr(p, shape)(x='x', y='y', size=16, line_width=2, color=color,
                                 source=computed_source, legend=d)
    # Legend
    p.legend.location = 'bottom_left'
    p.legend[0].orientation = 'horizontal'
    p.legend.label_text_font_size = '20pt'

    if name is None:
        show(p)
    else:
        p.output_backend = 'svg'
        export_svgs(p, name)
        return p


def plot_coverage_on_beta_cut(scoring='r2', black_box=None, draw_left_axis_label=True, draw_bottom_axis=True, name=None, legend_position='bottom_left', legend_orientation='vertical'):
    # Data gathering
    if isinstance(black_box, str):
        df = full_results[(full_results.black_box == black_box) & (full_results.scoring == scoring)\
                         & (full_results.alpha == .5)]
    else:
        df = full_results[(full_results.black_box.isin(black_box)) & (full_results.scoring == scoring)\
                         & (full_results.alpha == .5)]

    # Plot
    scoring = 'r²' if scoring == 'r2' else scoring
    p = figure(title='[' + scoring + '] Coverage [' + black_box.upper() + ']',
               x_axis_type='linear', x_minor_ticks=5, plot_width=600, plot_height=500)
    p.title.text_font_size = '20pt'

    # Axes
    p.xaxis[0].ticker.desired_num_ticks = 10
    p.yaxis[0].ticker.desired_num_ticks = 5
    p.xaxis[0].axis_label_text_font_size = '25pt'
    p.yaxis[0].axis_label_text_font_size = '25pt' if draw_left_axis_label else '0pt'
    p.xaxis[0].major_label_text_font_size = '20pt'
    p.yaxis[0].major_label_text_font_size = '20pt' if draw_left_axis_label else '0pt'
    p.xaxis.axis_label = 'β'
    # p.yaxis.visible = draw_left_axis_label
    p.yaxis.axis_label = 'coverage' if draw_left_axis_label else ''

    # Plot borders
    left, right, bottom, top = 0, 109, 0, 1.09
    p.x_range = Range1d(left, right)
    p.y_range = Range1d(bottom, top)

    for d, shape, color in zip(datasets, shapes, plt_colors4):
        dataset_df = df[df.dataset == d]
        
        # Fill nan data
        nan_index_start = np.argwhere(~dataset_df['beta-scoring-coverage'].notnull()).flatten().squeeze()
        nan_index_start = nan_index_start.item(0) if nan_index_start.size > 0 else dataset_df.shape[0]
        dataset_df.loc[~dataset_df['beta-scoring-coverage'].notnull(), 'beta-scoring-coverage'] = dataset_df[dataset_df['beta-scoring-coverage'].notnull()].iloc[-1]['beta-scoring-coverage']
        
        x, y = dataset_df.beta.values, dataset_df['beta-scoring-coverage'].values
        argmax_y = np.where(y == y.max())[0][-1]

        # Data labels
        source = ColumnDataSource(data=dict(x=x, y=y, names=[str(y_)[:4] for y_ in y]))
        computed_source = ColumnDataSource(data=dict(x=x, y=y))
        best_source = ColumnDataSource(data=dict(x=[x[argmax_y]], y=[y[argmax_y]]))

        # Available data
        line = p.line(x='x', y='y', line_width=4, alpha=.5,
                      line_color=color, source=computed_source, line_dash='dashed')
        dots = getattr(p, shape)(x='x', y='y', size=16, line_width=2, color=color,
                                 source=computed_source, legend=d)
    # Legend
    p.legend.location = legend_position
    p.legend[0].orientation = legend_orientation
    p.legend.label_text_font_size = '20pt'

    if name is None:
        show(p)
    else:
        p.output_backend = 'svg'
        export_svgs(p, name)
        return p


def plot_scorings_on_beta_cut(dataset, black_box, measure='fidelity', draw_left_axis_label=True, draw_bottom_axis=True, name=None, legend_position='bottom_left', legend_orientation='vertical'):
    # Data gathering
    if isinstance(black_box, str):
        df = full_results[(full_results.black_box == black_box) & (full_results.dataset == dataset)\
                         & (full_results.alpha == .5)]
        df = df.drop_duplicates()
    x = sorted(set(df.beta))

    # Plot
    p = figure(title='[' + dataset.capitalize() + '] [' + black_box.upper() + ']',
               x_axis_type='linear', x_minor_ticks=5, plot_width=600, plot_height=500)
    p.title.text_font_size = '20pt'

    # Axes
    p.xaxis[0].ticker.desired_num_ticks = 10
    p.yaxis[0].ticker.desired_num_ticks = 5
    p.xaxis[0].axis_label_text_font_size = '25pt'
    p.yaxis[0].axis_label_text_font_size = '25pt' if draw_left_axis_label else '0pt'
    p.xaxis[0].major_label_text_font_size = '20pt' if draw_bottom_axis else '0pt'
    p.yaxis[0].major_label_text_font_size = '20pt' if draw_left_axis_label else '0pt'
    p.xaxis.axis_label = 'β' if draw_bottom_axis else ''
    p.yaxis.axis_label = measure if draw_left_axis_label else ''

    # Plot borders
    left, right, bottom, top = 0, 109, 0, 1.09
    p.x_range = Range1d(left, right)
    p.y_range = Range1d(bottom, top)

    for candidate_measure, shape, color in zip(('fidelity', 'r2', 'coverage'), shapes, plt_colors3):
        data_df = df[df.scoring == candidate_measure]
        candidate_measure = 'r²' if candidate_measure == 'r2' else candidate_measure
        x = data_df['beta'].values
        y = data_df['beta-scoring-' + measure].values
            
        # x, y = data_df.index.values.tolist(), data_df['mean'].values
        if np.isnan(y).any():
            nan_index = np.argwhere(np.isnan(y)).squeeze().item(0)
            x, y = x[:nan_index], y[:nan_index]
            x = x[:y.shape[0]]
        argmax_y = np.where(y == y.max())[0][-1]
        
        # Data labels
        source = ColumnDataSource(data=dict(x=x, y=y, names=[str(y_)[:4] for y_ in y]))
        computed_source = ColumnDataSource(data=dict(x=x, y=y))
        best_source = ColumnDataSource(data=dict(x=[x[argmax_y]], y=[y[argmax_y]]))

        # Available data
        line = p.line(x='x', y='y', line_width=4, alpha=.5,
                      line_color=color, source=computed_source, line_dash='dashed')
        dots = getattr(p, shape)(x='x', y='y', size=16, line_width=2, color=color,
                                 source=computed_source, legend=candidate_measure[:3])
        dots = getattr(p, shape)(x='x', y='y', size=32, line_width=2, color=color, fill_alpha=0,
                                 source=best_source, legend=candidate_measure[:3])
    
    # Draw union baseline, i.e. cut on beta = 0
    df_prime = full_results[(full_results.black_box == black_box) & (full_results.dataset == dataset)
                            & (full_results.scoring == 'r2') & (full_results.beta == 0)]
    base_y = df_prime['scoring-fidelities'].values.item(0)
    # p.line(x=[left, right], y=[base_y, base_y], line_width=2, color=BLACK, alpha=.5, line_dash='dashed', legend='no pruning')
    
    # Legend
    p.legend.location = legend_position
    p.legend[0].orientation = legend_orientation
    p.legend.label_text_font_size = '20pt'

    if name is None:
        show(p)
    else:
        p.output_backend = 'svg'
        export_svgs(p, name)
        return p

## Plots

In [11]:
for d in datasets:
    bottom = d == 'german'
    for bb in ['dnn', 'rf']:
        for measure in ['fidelity', 'coverage']:
            left_axis = measure == 'fidelity'
            print(d, bb, measure)
            plot_scorings_on_beta_cut(d, bb, measure=measure,
                                      draw_left_axis_label=left_axis, draw_bottom_axis=bottom,
                                      name=None, legend_position='bottom_left', legend_orientation='horizontal')

# Black box by black_box
for bb in black_boxes:
    for cut in ['beta']:
        for scoring, left_axis in [('r2', True), ('coverage', False), ('fidelity', False)]:
            continue
            plot_fidelity_on_beta_cut(black_box=bb, scoring=scoring, name=None, draw_left_axis_label=left_axis)
            
for bb in black_boxes:
    for cut in ['beta']:
        for measure, scoring, left_axis in [('scoring', 'r2', True), ('coverage', 'coverage', False), ('fidelity', 'fidelity', False)]:            
            continue
            plot_coverage_on_beta_cut(black_box=bb, scoring=scoring, name=None, draw_left_axis_label=left_axis)

adult dnn fidelity
adult dnn coverage
adult rf fidelity
adult rf coverage
churn dnn fidelity
churn dnn coverage
churn rf fidelity
churn rf coverage
compas dnn fidelity
compas dnn coverage
compas rf fidelity
compas rf coverage
german dnn fidelity
german dnn coverage
german rf fidelity
german rf coverage


In [12]:
checkpoint('analysis')

## Adversaries

In [13]:
def plot_fidelity_on_beta_cut_adversaries(scoring='r2', black_box=None, draw_left_axis_label=True, name=None):
    # Data gathering
    if isinstance(black_box, str):
        df = full_adversaries_results[(full_adversaries_results.black_box == black_box) & (full_adversaries_results.scoring == scoring)\
                         & (full_adversaries_results.alpha == .5)]
    else:
        df = full_adversaries_results[(full_adversaries_results.black_box.isin(black_box)) & (full_adversaries_results.scoring == scoring)\
                         & (full_adversaries_results.alpha == .5)]

    # Plot
    scoring = 'r²' if scoring == 'r2' else scoring
    p = figure(title='[' + scoring + '] Fidelity [' + black_box.upper() + ']', x_axis_type='linear',
               x_minor_ticks=5, plot_width=600, plot_height=500)
    p.title.text_font_size = '20pt'

    # Axes
    p.xaxis[0].ticker.desired_num_ticks = 10
    p.yaxis[0].ticker.desired_num_ticks = 5
    p.xaxis[0].axis_label_text_font_size = '25pt'
    p.yaxis[0].axis_label_text_font_size = '25pt' if draw_left_axis_label else '0pt'
    p.xaxis[0].major_label_text_font_size = '20pt'
    p.yaxis[0].major_label_text_font_size = '20pt' if draw_left_axis_label else '0pt'
    p.xaxis.axis_label = 'β'
    # p.yaxis.visible = draw_left_axis_label
    p.yaxis.axis_label = 'fidelity' if draw_left_axis_label else ''

    # Plot borders
    left, right, bottom, top = 0, 104, .5, 1.09
    p.x_range = Range1d(left, right)
    p.y_range = Range1d(bottom, top)

    for d, shape, color in zip(datasets, shapes, plt_colors4):
        dataset_df = df[df.dataset == d]
        
        # Fill nan data
        nan_index_start = np.argwhere(~dataset_df['beta-scoring-fidelity'].notnull()).flatten().squeeze()
        nan_index_start = nan_index_start.item(0) if nan_index_start.size > 0 else dataset_df.shape[0]
        dataset_df.loc[~dataset_df['beta-scoring-fidelity'].notnull(), 'beta-scoring-fidelity'] = dataset_df[dataset_df['beta-scoring-fidelity'].notnull()].iloc[-1]['beta-scoring-fidelity']
        
        x, y = dataset_df.beta.values, dataset_df['beta-scoring-fidelity'].values
        argmax_y = np.where(y == y.max())[0][-1]

        # Data labels
        source = ColumnDataSource(data=dict(x=x, y=y, names=[str(y_)[:4] for y_ in y]))
        computed_source = ColumnDataSource(data=dict(x=x, y=y))
        best_source = ColumnDataSource(data=dict(x=[x[argmax_y]], y=[y[argmax_y]]))

        # Available data
        line = p.line(x='x', y='y', line_width=4, alpha=.5,
                      line_color=color, source=computed_source, line_dash='dashed')
        dots = getattr(p, shape)(x='x', y='y', size=16, line_width=2, color=color,
                                 source=computed_source, legend=d)
    # Legend
    p.legend.location = 'bottom_left'
    p.legend[0].orientation = 'horizontal'
    p.legend.label_text_font_size = '20pt'

    if name is None:
        show(p)
    else:
        p.output_backend = 'svg'
        export_svgs(p, name)
        return p


def plot_coverage_on_beta_cut_adversaries(scoring='r2', black_box=None, draw_left_axis_label=True, draw_bottom_axis=True, name=None, legend_position='bottom_left', legend_orientation='vertical'):
    # Data gathering
    if isinstance(black_box, str):
        df = full_adversaries_results[(full_adversaries_results.black_box == black_box) & (full_adversaries_results.scoring == scoring)\
                         & (full_adversaries_results.alpha == .5)]
    else:
        df = full_adversaries_results[(full_adversaries_results.black_box.isin(black_box)) & (full_adversaries_results.scoring == scoring)\
                         & (full_adversaries_results.alpha == .5)]

    # Plot
    scoring = 'r²' if scoring == 'r2' else scoring
    p = figure(title='[' + scoring + '] Coverage [' + black_box.upper() + ']',
               x_axis_type='linear', x_minor_ticks=5, plot_width=600, plot_height=500)
    p.title.text_font_size = '20pt'

    # Axes
    p.xaxis[0].ticker.desired_num_ticks = 10
    p.yaxis[0].ticker.desired_num_ticks = 5
    p.xaxis[0].axis_label_text_font_size = '25pt'
    p.yaxis[0].axis_label_text_font_size = '25pt' if draw_left_axis_label else '0pt'
    p.xaxis[0].major_label_text_font_size = '20pt'
    p.yaxis[0].major_label_text_font_size = '20pt' if draw_left_axis_label else '0pt'
    p.xaxis.axis_label = 'β'
    # p.yaxis.visible = draw_left_axis_label
    p.yaxis.axis_label = 'coverage' if draw_left_axis_label else ''

    # Plot borders
    left, right, bottom, top = 0, 109, 0, 1.09
    p.x_range = Range1d(left, right)
    p.y_range = Range1d(bottom, top)

    for d, shape, color in zip(datasets, shapes, plt_colors4):
        dataset_df = df[df.dataset == d]
        
        # Fill nan data
        nan_index_start = np.argwhere(~dataset_df['beta-scoring-coverage'].notnull()).flatten().squeeze()
        nan_index_start = nan_index_start.item(0) if nan_index_start.size > 0 else dataset_df.shape[0]
        dataset_df.loc[~dataset_df['beta-scoring-coverage'].notnull(), 'beta-scoring-coverage'] = dataset_df[dataset_df['beta-scoring-coverage'].notnull()].iloc[-1]['beta-scoring-coverage']
        
        x, y = dataset_df.beta.values, dataset_df['beta-scoring-coverage'].values
        argmax_y = np.where(y == y.max())[0][-1]

        # Data labels
        source = ColumnDataSource(data=dict(x=x, y=y, names=[str(y_)[:4] for y_ in y]))
        computed_source = ColumnDataSource(data=dict(x=x, y=y))
        best_source = ColumnDataSource(data=dict(x=[x[argmax_y]], y=[y[argmax_y]]))

        # Available data
        line = p.line(x='x', y='y', line_width=4, alpha=.5,
                      line_color=color, source=computed_source, line_dash='dashed')
        dots = getattr(p, shape)(x='x', y='y', size=16, line_width=2, color=color,
                                 source=computed_source, legend=d)
    # Legend
    p.legend.location = legend_position
    p.legend[0].orientation = legend_orientation
    p.legend.label_text_font_size = '20pt'

    if name is None:
        show(p)
    else:
        p.output_backend = 'svg'
        export_svgs(p, name)
        return p


def plot_scorings_on_beta_cut_adversaries(dataset, black_box, measure='fidelity', draw_left_axis_label=True, name=None, legend_position='bottom_left', legend_orientation='vertical'):
    # Data gathering
    if isinstance(black_box, str):
        df = full_adversaries_results[(full_adversaries_results.black_box == black_box) & (full_adversaries_results.dataset == dataset)\
                         & (full_adversaries_results.alpha == .5)]
        df = df.drop_duplicates()
    x = sorted(set(df.beta))

    # Plot
    p = figure(title='[' + dataset.capitalize() + '] [' + black_box.upper() + ']',
               x_axis_type='linear', x_minor_ticks=5, plot_width=600, plot_height=500)
    p.title.text_font_size = '20pt'

    # Axes
    p.xaxis[0].ticker.desired_num_ticks = 10
    p.yaxis[0].ticker.desired_num_ticks = 5
    p.xaxis[0].axis_label_text_font_size = '25pt'
    p.yaxis[0].axis_label_text_font_size = '25pt' if draw_left_axis_label else '0pt'
    p.xaxis[0].major_label_text_font_size = '20pt'
    p.yaxis[0].major_label_text_font_size = '20pt' if draw_left_axis_label else '0pt'
    p.xaxis.axis_label = 'β'
    p.yaxis.axis_label = measure

    # Plot borders
    left, right, bottom, top = 0, 109, .5, 1.09
    p.x_range = Range1d(left, right)
    p.y_range = Range1d(bottom, top)

    for candidate_measure, shape, color in zip(('fidelity', 'r2', 'coverage'), shapes, plt_colors3):
        data_df = df[df.scoring == candidate_measure]
        candidate_measure = 'r²' if candidate_measure == 'r2' else candidate_measure
        x = data_df['beta'].values
        y = data_df['beta-scoring-fidelity'].values
            
        # x, y = data_df.index.values.tolist(), data_df['mean'].values
        if np.isnan(y).any():
            nan_index = np.argwhere(np.isnan(y)).squeeze().item(0)
            x, y = x[:nan_index], y[:nan_index]
            x = x[:y.shape[0]]
        argmax_y = np.where(y == y.max())[0][-1]
        
        # Data labels
        source = ColumnDataSource(data=dict(x=x, y=y, names=[str(y_)[:4] for y_ in y]))
        computed_source = ColumnDataSource(data=dict(x=x, y=y))
        best_source = ColumnDataSource(data=dict(x=[x[argmax_y]], y=[y[argmax_y]]))

        # Available data
        line = p.line(x='x', y='y', line_width=4, alpha=.5,
                      line_color=color, source=computed_source, line_dash='dashed')
        dots = getattr(p, shape)(x='x', y='y', size=16, line_width=2, color=color,
                                 source=computed_source, legend=candidate_measure[:3])
        dots = getattr(p, shape)(x='x', y='y', size=32, line_width=2, color=color, fill_alpha=0,
                                 source=best_source, legend=candidate_measure[:3])
    
    # Draw union baseline, i.e. cut on beta = 0
    df_prime = full_adversaries_results[(full_adversaries_results.black_box == black_box) & (full_adversaries_results.dataset == dataset)
                            & (full_adversaries_results.scoring == 'r2') & (full_adversaries_results.beta == 0)]
    base_y = df_prime['scoring-fidelities'].values.item(0)
    p.line(x=[left, right], y=[base_y, base_y], line_width=2, color=BLACK, alpha=.5, line_dash='dashed',
           legend='no pruning')    
    
    # Legend
    p.legend.location = legend_position
    p.legend[0].orientation = legend_orientation
    p.legend.label_text_font_size = '20pt'

    if name is None:
        show(p)
    else:
        p.output_backend = 'svg'
        export_svgs(p, name)
        return p

## Plot rule size

In [14]:
from scipy.stats import hmean

def hmean_(a, b):
    return 0 if a == 0 or b == 0 else hmean([a, b])

hmeans = [hmean_(a, b) for a, b in full_adversaries_results[['beta-scoring-fidelity', 'beta-scoring-coverage']].values]
full_adversaries_results['beta-scoring-hmean'] = hmeans
hmeans = [hmean_(a, b) for a, b in full_results[['beta-scoring-fidelity', 'beta-scoring-coverage']].values]
full_results['beta-scoring-hmean'] = hmeans

draw_left_axis_label = True
legend_position = 'bottom_left'
legend_orientation = 'horizontal'
name = None

plt_colors3 = ['#009988', '#004488', '#ddaa33', '#bb5566', '#000000']
for bb in black_boxes:
    for dataset in datasets:        
        # Plot borders
        left, right, bottom, top = -9, 109, 0, 1
        for foo in ['rule_nr']:
            draw_left_axis_label = True

            # Plot
            p = figure(x_axis_type='linear', x_minor_ticks=5, plot_width=600, plot_height=400,
                      title=dataset.capitalize() + ' - ' + bb.upper())
            p.title.text_font_size = '25pt'

            # Axes
            p.xaxis[0].ticker.desired_num_ticks = 5
            p.yaxis[0].ticker.desired_num_ticks = 4
            p.xaxis[0].axis_label_text_font_size = '25pt'
            p.yaxis[0].axis_label_text_font_size = '25pt'
            p.xaxis[0].major_label_text_font_size = '25pt'
            p.yaxis[0].major_label_text_font_size = '25pt'
            p.xaxis.axis_label = 'β'
            p.yaxis.axis_label = 'size'
            p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
            p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
            p.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
            p.yaxis.minor_tick_line_color = None  # turn off y-axis minor ticks
            
            legend_entries = []
            
            # Draw union baseline, i.e. cut on beta = 0
            df_prime = full_adversaries_results[(full_adversaries_results.black_box == bb)\
                                                & (full_adversaries_results.dataset == dataset)\
                                                & (full_adversaries_results.scoring == 'r2')\
                                                & (full_adversaries_results.beta == 0)]
            base_y = df_prime['scoring-fidelities'].values.item(0)
            
            for adv, shape, color in zip(adversaries + ['r2'], shapes, plt_colors3):     
                if dataset == 'german' and adv == 'corels':
                    continue
                measure = foo
                x = [0, 25, 50, 75, 90, 99]
                if adv != 'r2':
                    df = full_adversaries_results[(full_adversaries_results.scoring == 'r2')\
                                                  & (full_adversaries_results.dataset == dataset)\
                                                  & (full_adversaries_results.black_box == bb)
                                                 & (full_adversaries_results.algorithm == adv)]\
                                                .groupby(['beta', 'algorithm'])\
                        [['beta-scoring-' + foo]].mean()
                    y = [df.loc[b, adv].values.item(0) for b in x]
                else:
                    df = full_results[(full_results.scoring == 'r2')\
                                      & (full_results.dataset == dataset)]\
                                        .groupby(['beta', 'black_box', 'dataset'])\
                        [['beta-scoring-' + foo]].mean()
                    y = [df.loc[b, bb, dataset].values.item(0) for b in x]
                y = array(y)
                
                # Ranges
                p.x_range = Range1d(left, right)
                p.y_range = Range1d(bottom, y.max())
                
                candidate_measure = 'r2'
                candidate_measure = 'r²' if candidate_measure == 'r2' else candidate_measure
                if np.isnan(y).any():
                    nan_index = np.argwhere(np.isnan(y)).squeeze().item(0)
                    x, y = x[:nan_index], y[:nan_index]
                    x = x[:y.shape[0]]
                argmax_y = np.where(y == y.max())[0][-1]

                # Data labels
                source = ColumnDataSource(data=dict(x=x, y=y, names=[str(y_)[:4] for y_ in y]))
                computed_source = ColumnDataSource(data=dict(x=x, y=y))
                best_source = ColumnDataSource(data=dict(x=[x[argmax_y]], y=[y[argmax_y]]))

                # Available data
                line = p.line(x='x', y='y', line_width=4, alpha=.5,
                              line_color=color, source=computed_source, line_dash='dashed')
                dots = getattr(p, shape)(x='x', y='y', size=16, line_width=2, color=color,
                                         source=computed_source)
                legend_entries.append((adv if adv != 'r2' else 'RRS', [dots]))
                
                if foo != 'coverage':
                    dots = getattr(p, shape)(x='x', y='y', size=32, line_width=2, color=color, fill_alpha=0,
                                             source=best_source)

            # Legend
            legend = Legend(items=legend_entries, location=(0, 0))
            legend.label_text_font_size = '20pt'
            legend.orientation = 'horizontal'
            legend.click_policy="mute"
            p.add_layout(legend, 'above')
            p.legend.glyph_height = 30
            p.legend.glyph_width = 30
            p.toolbar.logo = None
            p.toolbar_location = None

            show(p)

## Plot global measures (RRS vs competitors)

In [24]:
from scipy.stats import hmean

def hmean_(a, b):
    return 0 if a == 0 or b == 0 else hmean([a, b])

hmeans = [hmean_(a, b) for a, b in full_adversaries_results[['beta-scoring-fidelity', 'beta-scoring-coverage']].values]
full_adversaries_results['beta-scoring-hmean'] = hmeans
hmeans = [hmean_(a, b) for a, b in full_results[['beta-scoring-fidelity', 'beta-scoring-coverage']].values]
full_results['beta-scoring-hmean'] = hmeans

draw_left_axis_label = True
legend_position = 'bottom_left'
legend_orientation = 'horizontal'
name = None


plt_colors3 = ['#009988', '#004488', '#ddaa33', '#bb5566', '#000000']
for bb in black_boxes:
    for dataset in datasets:
        for foo in ['fidelity']:
            draw_left_axis_label = True

            # Plot
            p = figure(x_axis_type='linear', x_minor_ticks=5, plot_width=6000, plot_height=4000,
                      title=dataset.capitalize() + ' - ' + bb.upper())
            #p.output_backend = 'svg'
            p.title.text_font_size = '250pt'

            # Axes
            p.xaxis[0].ticker.desired_num_ticks = 5
            p.yaxis[0].ticker.desired_num_ticks = 6
            p.xaxis[0].axis_label_text_font_size = '250pt'
            p.yaxis[0].axis_label_text_font_size = '250pt'
            p.xaxis[0].major_label_text_font_size = '250pt'
            p.yaxis[0].major_label_text_font_size = '250pt'
            p.xaxis[0].axis_line_width *= 10
            p.yaxis[0].axis_line_width *= 10
            p.xaxis.axis_label = 'β'
            p.yaxis.axis_label = foo.capitalize()
            p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
            p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
            p.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
            p.yaxis.minor_tick_line_color = None  # turn off y-axis minor ticks
            p.yaxis[0].formatter = NumeralTickFormatter(format="0.0")
            
            legend_entries = []
            
            # Draw union baseline, i.e. cut on beta = 0
            df_prime = full_adversaries_results[(full_adversaries_results.black_box == bb)\
                                                & (full_adversaries_results.dataset == dataset)\
                                                & (full_adversaries_results.scoring == 'r2')\
                                                & (full_adversaries_results.beta == 0)]
            base_y = df_prime['scoring-fidelities'].values.item(0)

            for adv, shape, color in zip(adversaries + ['r2'], shapes, plt_colors3):     
                if dataset == 'german' and adv == 'corels':
                    continue
                measure = foo
                x = [0, 50, 75, 90, 99]
                if adv != 'r2':
                    df = full_adversaries_results[(full_adversaries_results.scoring == 'r2')\
                                                  & (full_adversaries_results.dataset == dataset)\
                                                  & (full_adversaries_results.black_box == bb)
                                                 & (full_adversaries_results.algorithm == adv)]\
                                                .groupby(['beta', 'algorithm'])\
                        [['beta-scoring-' + foo]].mean()
                    y = [df.loc[b, adv].values.item(0) for b in [0, 50, 75, 90, 99]]
                else:
                    df = full_results[(full_results.scoring == 'r2')\
                                      & (full_results.dataset == dataset)]\
                                        .groupby(['beta', 'black_box', 'dataset'])\
                        [['beta-scoring-' + foo]].mean()
                    y = [df.loc[b, bb, dataset].values.item(0) for b in [0, 50, 75, 90, 99]]
                y = array(y)

                # Plot borders
                left, right, bottom, top = -9, 109, -0.1, 1.1
                p.x_range = Range1d(left, right)
                p.y_range = Range1d(0.5, 1.09)

                candidate_measure = 'r2'
                candidate_measure = 'r²' if candidate_measure == 'r2' else candidate_measure
                if np.isnan(y).any():
                    nan_index = np.argwhere(np.isnan(y)).squeeze().item(0)
                    x, y = x[:nan_index], y[:nan_index]
                    x = x[:y.shape[0]]
                argmax_y = np.where(y == y.max())[0][-1]

                # Data labels
                source = ColumnDataSource(data=dict(x=x, y=y, names=[str(y_)[:4] for y_ in y]))
                computed_source = ColumnDataSource(data=dict(x=x, y=y))
                best_source = ColumnDataSource(data=dict(x=[x[argmax_y]], y=[y[argmax_y]]))

                # Available data
                line = p.line(x='x', y='y', line_width=40, alpha=.5,
                              line_color=color, source=computed_source, line_dash='dashed')
                dots = getattr(p, shape)(x='x', y='y', size=160, line_width=20, color=color,
                                         source=computed_source)
                legend_entries.append((adv.upper() if adv != 'r2' else 'RRS', [dots]))
                
                if foo != 'coverage':
                    dots = getattr(p, shape)(x='x', y='y', size=320, line_width=20, color=color, fill_alpha=0,
                                             source=best_source)

            # Legend
            legend = Legend(items=legend_entries, location=(0, 0))
            legend.label_text_font_size = '200pt'
            legend.orientation = 'horizontal'
            # legend.click_policy="mute"
            p.add_layout(legend, 'above')
            p.legend.glyph_height = 300
            p.legend.glyph_width = 300
            #p.toolbar.logo = None
            #p.toolbar_location = None
            
            #show(p)
            export_png(p, filename='global-' + dataset + '-' + bb + '.png')

In [16]:
from scipy.stats import hmean

def hmean_(a, b):
    return 0 if a == 0 or b == 0 else hmean([a, b])

hmeans = [hmean_(a, b) for a, b in full_results[['beta-scoring-fidelity', 'beta-scoring-coverage']].values]
full_results['beta-scoring-hmean'] = hmeans
hmeans = [hmean_(a, b) for a, b in full_results[['beta-scoring-fidelity', 'beta-scoring-coverage']].values]
full_results['beta-scoring-hmean'] = hmeans

draw_left_axis_label = True
legend_position = 'bottom_left'
legend_orientation = 'horizontal'
name = None
names = {'fidelity': 'FS', 'coverage': 'CS', 'r2': 'RRS'}

plt_colors3 = ['#009988', '#004488', '#bb5566', '#000000', '#ddaa33']
for bb in black_boxes:
    for dataset in datasets:
        for foo in ['fidelity', 'coverage', 'hmean']:
            draw_left_axis_label = foo == 'fidelity'

            # Plot
            p = figure(x_axis_type='linear', x_minor_ticks=5, plot_width=600, plot_height=400,
                      title=dataset.capitalize() + ' - ' + bb.upper())
            # p.output_backend = 'svg'
            p.title.text_font_size = '25pt'

            # Axes
            p.xaxis[0].ticker.desired_num_ticks = 5
            p.yaxis[0].ticker.desired_num_ticks = 5
            p.xaxis[0].axis_label_text_font_size = '25pt'
            p.yaxis[0].axis_label_text_font_size = '25pt'
            p.xaxis[0].major_label_text_font_size = '25pt'
            p.yaxis[0].major_label_text_font_size = '25pt'
            p.xaxis.axis_label = 'β'
            p.yaxis.axis_label = foo.capitalize()
            p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
            p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
            p.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
            p.yaxis.minor_tick_line_color = None  # turn off y-axis minor ticks
            p.yaxis[0].formatter = NumeralTickFormatter(format="0.0")
            
            legend_entries = []
            
            # Draw union baseline, i.e. cut on beta = 0
            df_prime = full_results[(full_results.black_box == bb)\
                                                & (full_results.dataset == dataset)\
                                                & (full_results.scoring == 'r2')\
                                                & (full_results.beta == 0)]
            base_y = df_prime['scoring-fidelities'].values.item(0)
            # p.line(x=[left, right], y=[base_y, base_y], line_width=2, color=BLACK, alpha=.5, line_dash='dashed', legend='no pruning')
            
            print(foo)
            for scoring, shape, color in zip(['coverage', 'fidelity', 'r2'], shapes, plt_colors3):
                measure = foo
                measure_name = names[scoring]
                x = [0, 25, 50, 75, 90, 99]
                df = full_results[(full_results.scoring == scoring)\
                                  & (full_results.dataset == dataset)\
                                  & (full_results.black_box == bb)].groupby('beta').mean()
                y = df['beta-scoring-' + foo].values

                # Plot borders
                left, right, bottom, top = -9, 109, -0.1, 1.09
                p.x_range = Range1d(left, right)
                p.y_range = Range1d(bottom, top)

                candidate_measure = 'r2'
                candidate_measure = 'r²' if candidate_measure == 'r2' else candidate_measure
                if np.isnan(y).any():
                    nan_index = np.argwhere(np.isnan(y)).squeeze().item(0)
                    x, y = x[:nan_index], y[:nan_index]
                    x = x[:y.shape[0]]
                argmax_y = np.where(y == y.max())[0][-1]

                # Data labels
                source = ColumnDataSource(data=dict(x=x, y=y, names=[str(y_)[:4] for y_ in y]))
                computed_source = ColumnDataSource(data=dict(x=x, y=y))
                best_source = ColumnDataSource(data=dict(x=[x[argmax_y]], y=[y[argmax_y]]))

                # Available data
                line = p.line(x='x', y='y', line_width=4, alpha=.5,
                              line_color=color, source=computed_source, line_dash='dashed')
                dots = getattr(p, shape)(x='x', y='y', size=16, line_width=2, color=color,
                                         source=computed_source, legend=measure_name)
                legend_entries.append((scoring, [dots]))
                
                if foo != 'coverage':
                    dots = getattr(p, shape)(x='x', y='y', size=32, line_width=2, color=color, fill_alpha=0,
                                             source=best_source)

            # Legend
            p.legend.label_text_font_size = '25pt'
            p.legend.orientation = 'vertical'
            p.legend.location = 'bottom_left'
            p.legend.background_fill_alpha = .5
            p.legend.glyph_height = 30
            p.legend.glyph_width = 30         
            p.toolbar.logo = None
            p.toolbar_location = None

            export_png(p, 'scoring-' + dataset + '-' + foo + '-' + bb + '.png')

fidelity
coverage
hmean
fidelity
coverage
hmean
fidelity
coverage
hmean
fidelity
coverage
hmean
fidelity
coverage
hmean
fidelity
coverage
hmean
fidelity
coverage
hmean
fidelity
coverage
hmean


In [17]:
from scipy.stats import hmean

def hmean_(a, b):
    return 0 if a == 0 or b == 0 else hmean([a, b])

hmeans = [hmean_(a, b) for a, b in full_adversaries_results[['beta-scoring-fidelity', 'beta-scoring-coverage']].values]
full_adversaries_results['beta-scoring-hmean'] = hmeans
hmeans = [hmean_(a, b) for a, b in full_results[['beta-scoring-fidelity', 'beta-scoring-coverage']].values]
full_results['beta-scoring-hmean'] = hmeans

draw_left_axis_label = True
legend_position = 'bottom_left'
legend_orientation = 'horizontal'
name = None

plt_colors3 = ['#009988', '#004488', '#ddaa33', '#bb5566', '#000000']
for bb in black_boxes:
    for dataset in datasets:
        top_ = full_adversaries_results[(full_adversaries_results.dataset == dataset)\
                                       & (full_adversaries_results.black_box == bb)]['beta-scoring-rule_nr'].values.max()
        top_ = max(top_, full_results[(full_results.scoring == 'r2') & (full_results.dataset == dataset)]['beta-scoring-rule_nr'].values.max())
        bottom_ = - (top_ / p.yaxis[0].ticker.desired_num_ticks + 1)
        top_ = top_ + (top_ / p.yaxis[0].ticker.desired_num_ticks - 1)
        
        
        # Plot borders
        left, right, bottom, top = -9, 109, bottom_, top_
        print('top')
        print(top)
        for foo in ['rule_nr']:
            draw_left_axis_label = True

            # Plot
            p = figure(x_axis_type='linear', x_minor_ticks=5, plot_width=600, plot_height=400,
                      title=dataset.capitalize() + ' - ' + bb.upper())
            p.title.text_font_size = '25pt'

            # Axes
            p.xaxis[0].ticker.desired_num_ticks = 5
            p.yaxis[0].ticker.desired_num_ticks = 4
            p.xaxis[0].axis_label_text_font_size = '25pt'
            p.yaxis[0].axis_label_text_font_size = '25pt'
            p.xaxis[0].major_label_text_font_size = '25pt'
            p.yaxis[0].major_label_text_font_size = '25pt'
            p.xaxis.axis_label = 'β'
            p.yaxis.axis_label = '#rules'
            p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
            p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
            p.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
            p.yaxis.minor_tick_line_color = None  # turn off y-axis minor ticks
            p.x_range = Range1d(left, right)
            p.y_range = Range1d(bottom, top)
            
            legend_entries = []
            
            # Draw union baseline, i.e. cut on beta = 0
            df_prime = full_adversaries_results[(full_adversaries_results.black_box == bb)\
                                                & (full_adversaries_results.dataset == dataset)\
                                                & (full_adversaries_results.scoring == 'r2')\
                                                & (full_adversaries_results.beta == 0)]
            base_y = df_prime['scoring-fidelities'].values.item(0)
            # p.line(x=[left, right], y=[base_y, base_y], line_width=2, color=BLACK, alpha=.5, line_dash='dashed', legend='no pruning')
            
            print(foo)
            for adv, shape, color in zip(adversaries + ['r2'], shapes, plt_colors3):     
                if dataset == 'german' and adv == 'corels':
                    continue
                measure = foo
                print(dataset, bb, adv)
                x = [0, 25, 50, 75, 90, 99]
                if adv != 'r2':
                    df = full_adversaries_results[(full_adversaries_results.scoring == 'r2')\
                                                  & (full_adversaries_results.dataset == dataset)\
                                                  & (full_adversaries_results.black_box == bb)
                                                 & (full_adversaries_results.algorithm == adv)]\
                                                .groupby(['beta', 'algorithm'])\
                        [['beta-scoring-' + foo]].mean()
                    y = [df.loc[b, adv].values.item(0) for b in x]
                else:
                    df = full_results[(full_results.scoring == 'r2')\
                                      & (full_results.dataset == dataset)]\
                                        .groupby(['beta', 'black_box', 'dataset'])\
                        [['beta-scoring-' + foo]].mean()
                    y = [df.loc[b, bb, dataset].values.item(0) for b in x]
                y = array(y)
                print(dataset, bb, y)
                
                candidate_measure = 'r2'
                candidate_measure = 'r²' if candidate_measure == 'r2' else candidate_measure
                if np.isnan(y).any():
                    nan_index = np.argwhere(np.isnan(y)).squeeze().item(0)
                    x, y = x[:nan_index], y[:nan_index]
                    x = x[:y.shape[0]]
                argmax_y = np.where(y == y.max())[0][-1]

                # Data labels
                source = ColumnDataSource(data=dict(x=x, y=y, names=[str(y_)[:4] for y_ in y]))
                computed_source = ColumnDataSource(data=dict(x=x, y=y))
                best_source = ColumnDataSource(data=dict(x=[x[argmax_y]], y=[y[argmax_y]]))

                # Available data
                line = p.line(x='x', y='y', line_width=4, alpha=.5,
                              line_color=color, source=computed_source, line_dash='dashed')
                dots = getattr(p, shape)(x='x', y='y', size=16, line_width=2, color=color,
                                         source=computed_source)
                legend_entries.append((adv.upper() if adv != 'r2' else 'RRS', [dots]))
                
                if foo != 'coverage':
                    dots = getattr(p, shape)(x='x', y='y', size=32, line_width=2, color=color, fill_alpha=0,
                                             source=best_source)

            # Legend
            legend = Legend(items=legend_entries, location=(0, 0))
            legend.label_text_font_size = '15pt'
            legend.orientation = 'horizontal'
            p.add_layout(legend, 'above')
            p.legend.glyph_height = 30
            p.legend.glyph_width = 30
            p.toolbar.logo = None
            p.toolbar_location = None
            
            #p.legend.location = 'top_left'
            #p.legend[0].orientation = legend_orientation
            #p.legend.label_text_font_size = '20pt'
            #p.legend.background_fill_alpha = 0.

            export_png(p, filename='size-' + dataset + '-' + foo + '-' + bb + '.png')

top
1728.2
rule_nr
adult dnn corels
adult dnn [1 1 1 1 1 1]
adult dnn cpar
adult dnn [299 224 150  75  30   3]
adult dnn sbrl
adult dnn [24 18 12  6  3  1]
adult dnn r2
adult dnn [1324  993  662  331  133   14]
top
275.25
rule_nr
churn dnn corels
churn dnn [3 2 2 1 1 1]
churn dnn cpar
churn dnn [94 71 48 24 12  2]
churn dnn sbrl
churn dnn [6 4 3 2 1 1]
churn dnn r2
churn dnn [201 151 101  51  21   4]
top
2807.75
rule_nr
compas dnn corels
compas dnn [1 1 1 1 1 1]
compas dnn cpar
compas dnn [2247 1686 1124  562  227   72]
compas dnn sbrl
compas dnn [7 7 7 2 1 1]
compas dnn r2
compas dnn [232 174 116  58  24   3]
top
146.5
rule_nr
german dnn cpar
german dnn [26 19 13  7  3  1]
german dnn sbrl
german dnn [2 2 2 2 2 2]
german dnn r2
german dnn [84 63 42 21  9  1]
top
1800.25
rule_nr
adult rf corels
adult rf [1 1 1 1 1 1]
adult rf cpar
adult rf [85 64 43 22  9  1]
adult rf sbrl
adult rf [26 20 13  7  3  1]
adult rf r2
adult rf [1441 1081  721  361  146   17]
top
275.25
rule_nr
churn rf corel

In [18]:
from scipy.stats import hmean

def hmean_(a, b):
    return 0 if a == 0 or b == 0 else hmean([a, b])

hmeans = [hmean_(a, b) for a, b in full_adversaries_results[['beta-scoring-fidelity', 'beta-scoring-coverage']].values]
full_adversaries_results['beta-scoring-hmean'] = hmeans
hmeans = [hmean_(a, b) for a, b in full_results[['beta-scoring-fidelity', 'beta-scoring-coverage']].values]
full_results['beta-scoring-hmean'] = hmeans
df1 = full_adversaries_results[(full_adversaries_results.scoring == 'r2')\
                               & (full_adversaries_results.beta == 0)].groupby(['dataset', 'black_box', 'algorithm']).mean()\
                                [['beta-scoring-rule_nr', 'mean-beta-scoring-length', 'beta-scoring-hmean']]
full_results['algorithm'] = 'scoring'
df2 = full_results[(full_results.scoring == 'r2')\
                   & (full_results.beta == 75)].groupby(['dataset', 'black_box', 'algorithm']).mean()\
                    [['beta-scoring-rule_nr', 'mean-beta-scoring-length', 'beta-scoring-hmean']]

df = pd.concat([df1.reset_index(), df2.reset_index()], ignore_index=True).sort_values(['dataset', 'black_box'])
df[(df.black_box == 'rf') & (df.algorithm == 'sbrl')][['beta-scoring-hmean', 'beta-scoring-rule_nr', 'mean-beta-scoring-length']]

,beta-scoring-hmean,beta-scoring-rule_nr,mean-beta-scoring-length
5,0.290613,26.0,1.0
11,0.246498,6.0,1.0
17,0.556272,5.0,1.0
21,0.474393,6.0,1.0


In [19]:
from scipy.stats import hmean

def hmean_(a, b):
    return 0 if a == 0 or b == 0 else hmean([a, b])

hmeans = [hmean_(a, b) for a, b in full_adversaries_results[['beta-scoring-fidelity', 'beta-scoring-coverage']].values]
full_adversaries_results['hmean'] = hmeans
hmeans = [hmean_(a, b) for a, b in full_results[['beta-scoring-fidelity', 'beta-scoring-coverage']].values]
full_results['hmean'] = hmeans

In [20]:
from more_itertools import flatten

dfs = []
for d in datasets:
    df = full_adversaries_results[(full_adversaries_results.dataset == d)\
                                  & (full_adversaries_results.black_box == 'dnn')]\
        .groupby(['algorithm', 'dataset', 'beta'])['beta-scoring-rule_nr'].describe()[['mean', 'std']]
    df = df.reset_index()
    dfs.append(df)

df = pd.concat(dfs, axis='columns', ignore_index=True)
df = df.drop([5, 7, 10, 12, 15, 17], axis='columns')
df.columns = ['Algorithm', 'dataset1', 'beta', 'mean1', 'std1',
              'dataset2', 'mean2', 'std2',
              'dataset3', 'mean3', 'std3',
              'dataset4', 'mean4', 'std4']
for d, i in zip(datasets, range(1, 5)):
    df[d + ' $\mu \pm \sigma$'] = str(df['mean' + str(i)]) + ' \pm ' + str(df['std' + str(i)])

df_1 = df[~df.dataset1.isin({'churn', 'compas', 'german'})][['Algorithm', 'beta', 'dataset1', 'mean1', 'std1', 'adult $\mu \pm \sigma$']]
df_2 = df[~df.dataset2.isin({'adult', 'compas', 'german'})][['Algorithm', 'beta', 'dataset2', 'mean2', 'std2', 'churn $\mu \pm \sigma$']]
df_3 = df[~df.dataset3.isin({'adult', 'churn', 'german'})][['Algorithm', 'beta', 'dataset3', 'mean3', 'std3', 'compas $\mu \pm \sigma$']]
df_4 = df[~df.dataset4.isin({'adult', 'churn', 'compas'})][['Algorithm', 'beta', 'dataset4', 'mean4', 'std4', 'german $\mu \pm \sigma$']]

df_1['$\mu \pm \sigma$'] = df_1[['mean1', 'std1']].apply(lambda x: '$' + str(x.mean1)[:4] + ' \pm '  + str(x.std1)[:4] + '$' if not np.isnan(x.mean1) else '-', axis='columns')
df_2['$\mu \pm \sigma$'] = df_2[['mean2', 'std2']].apply(lambda x: '$' + str(x.mean2)[:4] + ' \pm '  + str(x.std2)[:4] + '$' if not np.isnan(x.mean2) else '-', axis='columns')
df_3['$\mu \pm \sigma$'] = df_3[['mean3', 'std3']].apply(lambda x: '$' + str(x.mean3)[:4] + ' \pm '  + str(x.std3)[:4] + '$' if not np.isnan(x.mean3) else '-', axis='columns')
df_4['$\mu \pm \sigma$'] = df_4[['mean4', 'std4']].apply(lambda x: '$' + str(x.mean4)[:4] + ' \pm '  + str(x.std4)[:4] + '$' if not np.isnan(x.mean4) else '-', axis='columns')

df_1.columns = ['alg', '$\beta$', 'dataset', 'del1', 'del2', 'del3', '$\mu \pm \sigma$']
df_1 = df_1[['dataset', 'alg', '$\beta$', '$\mu \pm \sigma$']]

df_2.columns = ['alg', '$\beta$', 'dataset', 'del1', 'del2', 'del3', '$\mu \pm \sigma$']
df_2 = df_2[['dataset', 'alg', '$\beta$', '$\mu \pm \sigma$']]

df_3.columns = ['alg', '$\beta$', 'dataset', 'del1', 'del2', 'del3', '$\mu \pm \sigma$']
df_3 = df_3[['dataset', 'alg', '$\beta$', '$\mu \pm \sigma$']]

df_4.columns = ['alg', '$\beta$', 'dataset', 'del1', 'del2', 'del3', '$\mu \pm \sigma$']
df_4 = df_4[['dataset', 'alg', '$\beta$', '$\mu \pm \sigma$']]

df_tot = pd.concat([df_1, df_2, df_3, df_4], axis='columns', ignore_index=True)
df_tot.columns = list(flatten([['d{0}'.format(i), 'alg{0}'.format(i), 'beta{0}'.format(i), 'v{0}'.format(i)] for i in range(4)]))
df_tot = df_tot.drop(['alg1', 'alg2', 'alg3', 'beta1', 'beta2', 'beta3'], axis='columns')
df_tot.columns = ['alg', '$beta$', 'dataset', '$\mu \pm \sigma$', 'dataset', '$\mu \pm \sigma$', 'dataset', '$\mu \pm \sigma$', 'dataset', '$\mu \pm \sigma$']

table = str(df_tot.to_latex(escape=False))
table = table.replace('toprule', 'hline').replace('midrule', 'hline').replace('bottomrule', 'hline')
table = table.replace('0.00 &', '0 &').replace('25.00 &', '25 &').replace('50.00 &', '50 &').replace('75.00 &', '75 &').replace('90.00 &', '90 &').replace('99.00 &', '99 &')
table = table.replace('beta', '\beta')
print(table)
df_tot

\begin{tabular}{lllrlllllll}
\hline
{} &    alg &  $eta$ & dataset & $\mu \pm \sigma$ & dataset & $\mu \pm \sigma$ & dataset & $\mu \pm \sigma$ & dataset & $\mu \pm \sigma$ \\
\hline
0  &  adult &  corels &     0.0 &    $1.0 \pm 0.0$ &   churn &    $3.0 \pm 0.0$ &  compas &    $1.0 \pm 0.0$ &  german &   $26.0 \pm 0.0$ \\
1  &  adult &  corels &    25.0 &    $1.0 \pm 0.0$ &   churn &    $2.0 \pm 0.0$ &  compas &    $1.0 \pm 0.0$ &  german &   $19.0 \pm 0.0$ \\
2  &  adult &  corels &    50.0 &    $1.0 \pm 0.0$ &   churn &    $2.0 \pm 0.0$ &  compas &    $1.0 \pm 0.0$ &  german &   $13.0 \pm 0.0$ \\
3  &  adult &  corels &    75.0 &    $1.0 \pm 0.0$ &   churn &   $1.33 \pm 0.5$ &  compas &    $1.0 \pm 0.0$ &  german &  $8.05 \pm 1.45$ \\
4  &  adult &  corels &    90.0 &    $1.0 \pm 0.0$ &   churn &   $1.33 \pm 0.5$ &  compas &    $1.0 \pm 0.0$ &  german &  $5.62 \pm 3.44$ \\
5  &  adult &  corels &    99.0 &    $1.0 \pm 0.0$ &   churn &   $1.33 \pm 0.5$ &  compas &    $1.0 \pm 0.0$ & 

,alg,$beta$,dataset,$\mu \pm \sigma$,dataset,$\mu \pm \sigma$,dataset,$\mu \pm \sigma$,dataset,$\mu \pm \sigma$
0,adult,corels,0.0,$1.0 \pm 0.0$,churn,$3.0 \pm 0.0$,compas,$1.0 \pm 0.0$,german,$26.0 \pm 0.0$
1,adult,corels,25.0,$1.0 \pm 0.0$,churn,$2.0 \pm 0.0$,compas,$1.0 \pm 0.0$,german,$19.0 \pm 0.0$
2,adult,corels,50.0,$1.0 \pm 0.0$,churn,$2.0 \pm 0.0$,compas,$1.0 \pm 0.0$,german,$13.0 \pm 0.0$
3,adult,corels,75.0,$1.0 \pm 0.0$,churn,$1.33 \pm 0.5$,compas,$1.0 \pm 0.0$,german,$8.05 \pm 1.45$
4,adult,corels,90.0,$1.0 \pm 0.0$,churn,$1.33 \pm 0.5$,compas,$1.0 \pm 0.0$,german,$5.62 \pm 3.44$
5,adult,corels,99.0,$1.0 \pm 0.0$,churn,$1.33 \pm 0.5$,compas,$1.0 \pm 0.0$,german,$4.85 \pm 4.53$
6,adult,cpar,0.0,$299. \pm 0.0$,churn,$94.0 \pm 0.0$,compas,$2247 \pm 0.0$,german,$2.0 \pm 0.0$
7,adult,cpar,25.0,$241. \pm 26.5$,churn,$73.2 \pm 2.08$,compas,$1691 \pm 6.71$,german,$2.0 \pm 0.0$
8,adult,cpar,50.0,$155. \pm 4.48$,churn,$50.8 \pm 2.46$,compas,$1124 \pm 0.47$,german,$2.0 \pm 0.0$
9,adult,cpar,75.0,$92.6 \pm 24.3$,churn,$27.2 \pm 4.35$,compas,$565. \pm 4.30$,german,$2.0 \pm 0.0$


In [21]:
for d in [df_1, df_2, df_3, df_4]:
    print(d.iloc[:, -1].to_latex(escape=False))

\begin{tabular}{ll}
\toprule
{} & $\mu \pm \sigma$ \\
\midrule
0  &    $1.0 \pm 0.0$ \\
1  &    $1.0 \pm 0.0$ \\
2  &    $1.0 \pm 0.0$ \\
3  &    $1.0 \pm 0.0$ \\
4  &    $1.0 \pm 0.0$ \\
5  &    $1.0 \pm 0.0$ \\
6  &   $299. \pm 0.0$ \\
7  &  $241. \pm 26.5$ \\
8  &  $155. \pm 4.48$ \\
9  &  $92.6 \pm 24.3$ \\
10 &  $45.4 \pm 21.0$ \\
11 &    $3.0 \pm 0.0$ \\
12 &   $24.0 \pm 0.0$ \\
13 &   $18.0 \pm 0.0$ \\
14 &   $12.0 \pm 0.0$ \\
15 &   $7.0 \pm 1.45$ \\
16 &   $5.0 \pm 2.91$ \\
17 &    $4.0 \pm 4.0$ \\
\bottomrule
\end{tabular}

\begin{tabular}{ll}
\toprule
{} & $\mu \pm \sigma$ \\
\midrule
0  &    $3.0 \pm 0.0$ \\
1  &    $2.0 \pm 0.0$ \\
2  &    $2.0 \pm 0.0$ \\
3  &   $1.33 \pm 0.5$ \\
4  &   $1.33 \pm 0.5$ \\
5  &   $1.33 \pm 0.5$ \\
6  &   $94.0 \pm 0.0$ \\
7  &  $73.2 \pm 2.08$ \\
8  &  $50.8 \pm 2.46$ \\
9  &  $27.2 \pm 4.35$ \\
10 &  $19.8 \pm 10.2$ \\
11 &  $14.5 \pm 15.3$ \\
12 &    $6.0 \pm 0.0$ \\
13 &    $4.0 \pm 0.0$ \\
14 &    $3.0 \pm 0.0$ \\
15 &  $2.33 \pm 0.49$ 

In [22]:
from more_itertools import flatten

dfs = []
for d in datasets:
    df = full_adversaries_results[(full_adversaries_results.dataset == d)\
                                  & (full_adversaries_results.black_box == 'rf')]\
        .groupby(['algorithm', 'dataset', 'beta'])['beta-scoring-rule_nr'].describe()[['mean', 'std']]
    df = df.reset_index()
    print(df.shape)
    dfs.append(df)

df = pd.concat(dfs, axis='columns', ignore_index=True)
df = df.drop([5, 7, 10, 12, 15, 17], axis='columns')
df.columns = ['Algorithm', 'dataset1', 'beta', 'mean1', 'std1',
              'dataset2', 'mean2', 'std2',
              'dataset3', 'mean3', 'std3',
              'dataset4', 'mean4', 'std4']

for d, i in zip(datasets, range(1, 5)):
    df[d + ' $\mu \pm \sigma$'] = str(df['mean' + str(i)]) + ' \pm ' + str(df['std' + str(i)])

df_1 = df[~df.dataset1.isin({'churn', 'compas', 'german'})][['Algorithm', 'beta', 'dataset1', 'mean1', 'std1', 'adult $\mu \pm \sigma$']]
df_2 = df[~df.dataset2.isin({'adult', 'compas', 'german'})][['Algorithm', 'beta', 'dataset2', 'mean2', 'std2', 'churn $\mu \pm \sigma$']]
df_3 = df[~df.dataset3.isin({'adult', 'churn', 'german'})][['Algorithm', 'beta', 'dataset3', 'mean3', 'std3', 'compas $\mu \pm \sigma$']]
df_4 = df[~df.dataset4.isin({'adult', 'churn', 'compas'})][['Algorithm', 'beta', 'dataset4', 'mean4', 'std4', 'german $\mu \pm \sigma$']]

print(df_1.shape)
print(df_2.shape)
print(df_3.shape)
print(df_4.shape)

df_1['$\mu \pm \sigma$'] = df_1[['mean1', 'std1']].apply(lambda x: '$' + str(x.mean1)[:4] + ' \pm '  + str(x.std1)[:4] + '$' if not np.isnan(x.mean1) else '-', axis='columns')
df_2['$\mu \pm \sigma$'] = df_2[['mean2', 'std2']].apply(lambda x: '$' + str(x.mean2)[:4] + ' \pm '  + str(x.std2)[:4] + '$' if not np.isnan(x.mean2) else '-', axis='columns')
df_3['$\mu \pm \sigma$'] = df_3[['mean3', 'std3']].apply(lambda x: '$' + str(x.mean3)[:4] + ' \pm '  + str(x.std3)[:4] + '$' if not np.isnan(x.mean3) else '-', axis='columns')
df_4['$\mu \pm \sigma$'] = df_4[['mean4', 'std4']].apply(lambda x: '$' + str(x.mean4)[:4] + ' \pm '  + str(x.std4)[:4] + '$' if not np.isnan(x.mean4) else '-', axis='columns')

print(df_1.shape)
print(df_2.shape)
print(df_3.shape)
print(df_4.shape)

df_1.columns = ['alg', '$\beta$', 'dataset', 'del1', 'del2', 'del3', '$\mu \pm \sigma$']
df_1 = df_1[['dataset', 'alg', '$\beta$', '$\mu \pm \sigma$']]

df_2.columns = ['alg', '$\beta$', 'dataset', 'del1', 'del2', 'del3', '$\mu \pm \sigma$']
df_2 = df_2[['dataset', 'alg', '$\beta$', '$\mu \pm \sigma$']]

df_3.columns = ['alg', '$\beta$', 'dataset', 'del1', 'del2', 'del3', '$\mu \pm \sigma$']
df_3 = df_3[['dataset', 'alg', '$\beta$', '$\mu \pm \sigma$']]

df_4.columns = ['alg', '$\beta$', 'dataset', 'del1', 'del2', 'del3', '$\mu \pm \sigma$']
df_4 = df_4[['dataset', 'alg', '$\beta$', '$\mu \pm \sigma$']]

print(df_1.shape)
print(df_2.shape)
print(df_3.shape)
print(df_4.shape)

df_tot = pd.concat([df_1, df_2, df_3, df_4], axis='columns', ignore_index=True)
df_tot.columns = list(flatten([['d{0}'.format(i), 'alg{0}'.format(i), 'beta{0}'.format(i), 'v{0}'.format(i)] for i in range(4)]))
df_tot = df_tot.drop(['alg1', 'alg2', 'alg3', 'beta1', 'beta2', 'beta3'], axis='columns')
df_tot.columns = ['alg', '$beta$', 'dataset', '$\mu \pm \sigma$', 'dataset', '$\mu \pm \sigma$', 'dataset', '$\mu \pm \sigma$', 'dataset', '$\mu \pm \sigma$']

table = str(df_tot.to_latex(escape=False))
table = table.replace('toprule', 'hline').replace('midrule', 'hline').replace('bottomrule', 'hline')
table = table.replace('0.00 &', '0 &').replace('25.00 &', '25 &').replace('50.00 &', '50 &').replace('75.00 &', '75 &').replace('90.00 &', '90 &').replace('99.00 &', '99 &')
table = table.replace('beta', '\beta')
# print(table)
# df_tot

(18, 5)
(18, 5)
(18, 5)
(12, 5)
(18, 6)
(18, 6)
(18, 6)
(18, 6)
(18, 7)
(18, 7)
(18, 7)
(18, 7)
(18, 4)
(18, 4)
(18, 4)
(18, 4)


In [23]:
for d in [df_1, df_2, df_3, df_4]:
    print(d.iloc[:, -1].to_latex(escape=False))

\begin{tabular}{ll}
\toprule
{} & $\mu \pm \sigma$ \\
\midrule
0  &    $1.0 \pm 0.0$ \\
1  &    $1.0 \pm 0.0$ \\
2  &    $1.0 \pm 0.0$ \\
3  &    $1.0 \pm 0.0$ \\
4  &    $1.0 \pm 0.0$ \\
5  &    $1.0 \pm 0.0$ \\
6  &   $85.0 \pm 0.0$ \\
7  &  $69.6 \pm 9.40$ \\
8  &  $44.3 \pm 1.23$ \\
9  &   $22.0 \pm 0.0$ \\
10 &    $9.0 \pm 0.0$ \\
11 &    $1.0 \pm 0.0$ \\
12 &   $26.0 \pm 0.0$ \\
13 &  $19.3 \pm 0.48$ \\
14 &   $13.0 \pm 0.0$ \\
15 &  $8.33 \pm 1.95$ \\
16 &  $5.66 \pm 3.90$ \\
17 &  $4.33 \pm 4.87$ \\
\bottomrule
\end{tabular}

\begin{tabular}{ll}
\toprule
{} & $\mu \pm \sigma$ \\
\midrule
0  &    $1.0 \pm 0.0$ \\
1  &    $1.0 \pm 0.0$ \\
2  &    $1.0 \pm 0.0$ \\
3  &    $1.0 \pm 0.0$ \\
4  &    $1.0 \pm 0.0$ \\
5  &    $1.0 \pm 0.0$ \\
6  &   $130. \pm 0.0$ \\
7  &  $103. \pm 8.95$ \\
8  &  $71.3 \pm 7.94$ \\
9  &  $37.2 \pm 3.63$ \\
10 &  $18.8 \pm 8.17$ \\
11 &  $5.33 \pm 4.76$ \\
12 &    $6.0 \pm 0.0$ \\
13 &  $5.57 \pm 0.53$ \\
14 &  $5.57 \pm 0.53$ \\
15 &  $5.57 \pm 0.53$ 